In [14]:
#Library for mathematical computation
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
from time import time

#Pre-processamento
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import explained_variance_score,mean_absolute_error,r2_score
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import LabelEncoder #Encodes Categorical Data into Numerical

from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.model_selection import RepeatedKFold

from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
dataset=pd.read_csv("/content/drive/MyDrive/INF1032 - Spotify/Dados/Consolidados/global_final.csv")
dataset.columns

Index(['Track_id', 'Position', 'Streams', 'Start_date', 'Track Name', 'Artist',
       'Region', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url',
       'duration_ms', 'time_signature', 'artist_pop', 'artist_followers',
       'genres'],
      dtype='object')

In [17]:
dataset["mean_streams"] = dataset.Streams.apply(lambda x : sum(list(map(int, x.strip("[]").split(","))))/len(x.strip("[]").split(",")))
dataset["first_genre"] = dataset.genres.apply(lambda x : x.split(",", maxsplit=1)[0])
dataset["Highest_position"] = dataset["Position"].apply(lambda x : min(json.loads(x)))
dataset["N_times_charted"] = dataset["Position"].apply(lambda x : len(x))
dataset

,Track_id,Position,Streams,Start_date,Track Name,Artist,Region,danceability,energy,key,...,analysis_url,duration_ms,time_signature,artist_pop,artist_followers,genres,mean_streams,first_genre,Highest_position,N_times_charted
0,000xQL6tZNLJzIrtIgxqSl,"[46, 50, 52, 63, 57, 57, 60, 68, 74, 79, 109, ...","[10037227, 9573086, 8605305, 7663964, 8368114,...","['2017-03-24', '2017-03-31', '2017-04-07', '20...",Still Got Time (feat. PARTYNEXTDOOR),ZAYN,Global,0.748,0.627,7,...,https://api.spotify.com/v1/audio-analysis/000x...,188491,4,75,20323510,"dance_pop,electropop,pop,post-teen_pop,uk_pop",6.325162e+06,dance_pop,46,75
1,003eoIwxETJujVWmNFMoZy,"[112, 155]","[5586774, 4358091]","['2018-06-15', '2018-06-22']",Growing Pains,Alessia Cara,Global,0.353,0.755,1,...,https://api.spotify.com/v1/audio-analysis/003e...,193680,4,73,10388115,"canadian_contemporary_r&b,canadian_pop,dance_p...",4.972432e+06,canadian_contemporary_r&b,112,10
2,003vvx7Niy0yvhvHt4a68B,"[134, 186, 189, 164, 184, 176, 193, 153, 197, ...","[6158877, 5072849, 5119923, 5387795, 5259604, ...","['2022-05-06', '2021-02-26', '2021-03-12', '20...",Mr. Brightside,The Killers,Global,0.352,0.911,1,...,https://api.spotify.com/v1/audio-analysis/003v...,222973,4,76,6545714,"alternative_rock,dance_rock,modern_rock,perman...",5.588138e+06,alternative_rock,98,259
3,00B7TZ0Xawar6NZ00JFomN,"[75, 140]","[7241438, 4743908]","['2018-04-06', '2018-04-13']",Best Life (feat. Chance The Rapper),Cardi B,Global,0.620,0.625,9,...,https://api.spotify.com/v1/audio-analysis/00B7...,284856,4,80,20203300,"dance_pop,pop,pop_rap,rap",5.992673e+06,dance_pop,75,9
4,00Blm7zeNqgYLPtW6zg8cj,"[84, 24, 45, 52, 58, 9, 14, 64, 22, 32, 42, 48...","[7290094, 15011719, 12482575, 12040044, 111576...","['2022-05-06', '2021-11-26', '2021-12-03', '20...",One Right Now (with The Weeknd),Post Malone,Global,0.687,0.781,1,...,https://api.spotify.com/v1/audio-analysis/00Bl...,193507,4,89,37190511,"dfw_rap,melodic_rap,rap",1.053458e+07,dfw_rap,9,103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5151,7zVCrzzEJU7u24sbJPXA5W,"[174, 146, 164, 177]","[3614088, 4028775, 3764659, 3488593]","['2017-10-06', '2017-10-13', '2017-10-20', '20...",What Ifs (feat. Lauren Alaina),Kane Brown,Global,0.612,0.799,10,...,https://api.spotify.com/v1/audio-analysis/7zVC...,188253,4,73,4138637,"black_americana,contemporary_country,country_road",3.724029e+06,black_americana,146,20
5152,7zgqtptZvhf8GEmdsM2vp2,[13],[17583532],['2017-09-01'],...Ready For It?,Taylor Swift,Global,0.615,0.779,2,...,https://api.spotify.com/v1/audio-analysis/7zgq...,208198,4,92,54239405,pop,1.758353e+07,pop,13,4
5153,7zjEyeBsaw9gV0jofJLfOM,"[198, 191]","[5223949, 5302686]","['2021-10-22', '2021-10-15']",Livin It Up (with Post Malone & A$AP Rocky),Young Thug,Global,0.767,0.313,7,...,https://api.spotify.com/v1/audio-analysis/7zjE...,210907,4,84,7280318,"atl_hip_hop,atl_trap,gangster_rap,hip_hop,melo...",5.263318e+06,atl_hip_hop,191,10
5154,7zl7kehxesNEo2pYkKXTSe,[170],[5404939],['2021-05-28'],Killer (feat. Jack Harlow & Cordae) - Remix,Eminem,Global,0.924,0.730,11,...,https://api.spotify.com/v1/audio-analysis/7zl7...,236666,4,89,56456639,"detroit_hip_hop,hip_hop,rap",5.404939e+06,detroit_hip_hop,170,5


In [18]:
genre_encoder = LabelEncoder()
dataset['encoded_genre'] = genre_encoder.fit_transform(dataset['first_genre'])
dataset_cln = dataset.select_dtypes([np.number])
dataset_cln

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,artist_pop,artist_followers,mean_streams,Highest_position,N_times_charted,encoded_genre
0,0.748,0.627,7,-6.029,1,0.0639,0.13100,0.0,0.0852,0.524,120.963,188491,4,75,20323510,6.325162e+06,46,75,102
1,0.353,0.755,1,-6.276,0,0.7330,0.08220,0.0,0.3900,0.437,191.153,193680,4,73,10388115,4.972432e+06,112,10,63
2,0.352,0.911,1,-5.230,1,0.0747,0.00121,0.0,0.0995,0.236,148.033,222973,4,76,6545714,5.588138e+06,98,259,18
3,0.620,0.625,9,-7.438,1,0.5530,0.28700,0.0,0.3140,0.665,167.911,284856,4,80,20203300,5.992673e+06,75,9,102
4,0.687,0.781,1,-4.806,1,0.0530,0.03610,0.0,0.0755,0.688,97.014,193507,4,89,37190511,1.053458e+07,9,103,114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5151,0.612,0.799,10,-4.603,1,0.0275,0.00898,0.0,0.1720,0.687,125.976,188253,4,73,4138637,3.724029e+06,146,20,46
5152,0.615,0.779,2,-6.454,1,0.1350,0.06650,0.0,0.1550,0.453,160.000,208198,4,92,54239405,1.758353e+07,13,4,244
5153,0.767,0.313,7,-12.059,1,0.0798,0.83800,0.0,0.1050,0.765,82.582,210907,4,84,7280318,5.263318e+06,191,10,24
5154,0.924,0.730,11,-6.125,0,0.2740,0.05250,0.0,0.1130,0.848,99.010,236666,4,89,56456639,5.404939e+06,170,5,113


In [19]:
training_data, testing_data = train_test_split(dataset_cln, test_size=0.2, random_state=25)

In [20]:
x_test=testing_data.drop(columns=['mean_streams'],axis=1)
y_test=testing_data['mean_streams']

testing_data

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,artist_pop,artist_followers,mean_streams,Highest_position,N_times_charted,encoded_genre
1473,0.791,0.746,6,-4.716,1,0.0907,0.00938,0.000117,0.1250,0.905,165.994,183160,4,87,25362893,4.840329e+06,136,15,208
3158,0.612,0.862,8,-2.903,1,0.0370,0.04100,0.000000,0.0907,0.354,123.849,160766,4,76,477720,5.371262e+06,162,15,102
47,0.752,0.543,1,-9.483,1,0.0398,0.62300,0.070600,0.1070,0.282,126.001,233280,4,64,3848258,2.692337e+06,200,5,120
3303,0.448,0.612,2,-6.106,1,0.0708,0.21600,0.000000,0.0800,0.213,97.773,189467,4,86,40577557,1.073231e+07,20,64,225
2984,0.563,0.756,0,-3.852,0,0.0579,0.19600,0.000000,0.2450,0.234,100.012,182227,4,76,5893181,6.218121e+06,131,5,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4173,0.316,0.630,5,-5.963,1,0.1170,0.47100,0.262000,0.2050,0.313,89.643,153294,1,79,2386227,6.621249e+06,53,93,181
3168,0.772,0.504,6,-8.219,0,0.0761,0.12900,0.000000,0.1250,0.244,140.035,237573,4,91,45760384,3.907657e+06,104,40,63
3575,0.587,0.267,8,-9.206,1,0.0349,0.84000,0.000000,0.1210,0.352,98.845,185853,4,90,98436130,8.531058e+06,62,4,244
455,0.671,0.916,6,-4.014,0,0.0397,0.02820,0.007620,0.2400,0.803,126.010,189127,4,74,3236891,3.061179e+06,131,20,102


In [21]:
#Setting training data into x_train and y_train
x_train=training_data.drop('mean_streams',axis=1)
y_train=training_data['mean_streams']

training_data

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,artist_pop,artist_followers,mean_streams,Highest_position,N_times_charted,encoded_genre
660,0.905,0.686,0,-3.152,1,0.0664,0.0907,0.000000,0.2660,0.963,103.013,155825,4,61,1057448,9.206356e+06,24,108,209
135,0.572,0.691,8,-4.783,0,0.1230,0.0788,0.000132,0.0593,0.318,92.082,212040,4,88,34441936,4.878363e+06,134,25,102
3629,0.792,0.620,8,-4.832,0,0.2660,0.1100,0.000016,0.0850,0.637,169.985,175907,4,62,1089495,4.526638e+06,136,10,147
3879,0.655,0.672,0,-5.980,0,0.3110,0.4710,0.000000,0.0767,0.343,181.877,249520,4,75,4162635,8.040587e+06,29,158,208
3850,0.634,0.285,9,-14.007,0,0.0807,0.9370,0.000359,0.2320,0.203,129.650,245897,4,86,63888231,1.325251e+07,4,41,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1175,0.761,0.672,1,-5.887,1,0.2120,0.0893,0.000010,0.1400,0.437,160.077,192405,3,71,6726175,7.124780e+06,26,27,137
255,0.526,0.520,6,-7.985,0,0.0569,0.4350,0.000000,0.1970,0.130,149.119,183637,4,76,15565432,7.239705e+06,42,85,102
2934,0.861,0.647,7,-7.425,1,0.0675,0.5940,0.661000,0.1140,0.670,158.037,133680,4,81,9543238,4.615538e+06,180,10,17
2191,0.746,0.450,7,-8.543,1,0.0872,0.0407,0.000000,0.1720,0.336,95.998,232853,4,77,5057459,8.635144e+06,27,30,102


In [22]:
#Shapes of x_train,y_train and test data
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((4124, 18), (4124,), (1032, 18), (1032,))

In [23]:
regressors = [
    KNeighborsRegressor(),
    GradientBoostingRegressor(),
    KNeighborsRegressor(),
    ExtraTreesRegressor(),
    RandomForestRegressor(),
    DecisionTreeRegressor(),
    LinearRegression(),
    Lasso()
]

In [24]:
head = 10
for model in regressors[:head]:
    start = time()
    model.fit(x_train, y_train)
    train_time = time() - start
    start = time()
    y_pred = model.predict(x_test)
    predict_time = time()-start    
    print(model)
    print("\tTraining time: %0.3fs" % train_time)
    print("\tPrediction time: %0.3fs" % predict_time)
    print("\tExplained variance:", explained_variance_score(y_test, y_pred))
    print("\tR2 score:", r2_score(y_test, y_pred))
    print("\tmodel.score nos dados de teste:", model.score(x_test,y_test))
    print("\tmodel.score nos dados de treino:", model.score(x_train,y_train))
    print()

KNeighborsRegressor()
	Training time: 0.003s
	Prediction time: 0.126s
	Explained variance: -0.006564902835342279
	R2 score: -0.006832084354858159
	model.score nos dados de teste: -0.006832084354858159
	model.score nos dados de treino: 0.3483828596081995

GradientBoostingRegressor()
	Training time: 1.549s
	Prediction time: 0.004s
	Explained variance: 0.8446825325917771
	R2 score: 0.8444740612063215
	model.score nos dados de teste: 0.8444740612063215
	model.score nos dados de treino: 0.9313777577619141

KNeighborsRegressor()
	Training time: 0.003s
	Prediction time: 0.109s
	Explained variance: -0.006564902835342279
	R2 score: -0.006832084354858159
	model.score nos dados de teste: -0.006832084354858159
	model.score nos dados de treino: 0.3483828596081995

ExtraTreesRegressor()
	Training time: 1.854s
	Prediction time: 0.044s
	Explained variance: 0.8020846601243161
	R2 score: 0.8020843489159385
	model.score nos dados de teste: 0.8020843489159385
	model.score nos dados de treino: 0.9999999158